In [19]:
import pandas as pd

train = pd.read_csv(r"train_final.csv")

print(f'train set은 {train.shape[1]} 개의 feature를 가진 {train.shape[0]} 개의 데이터 샘플로 이루어져 있습니다.')

train.head()

train set은 24 개의 feature를 가진 2999 개의 데이터 샘플로 이루어져 있습니다.


,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,target
0,1,Male,Loyal Customer,72,Business travel,Business,1784,1,1,4,...,2,2,3,2,5,2,0,0,0,1
1,2,Male,Loyal Customer,35,Business travel,Business,2228,2,5,5,...,2,2,2,2,4,2,0,0,8,0
2,3,Male,disloyal Customer,37,Business travel,Eco,1970,4,3,4,...,1,3,2,5,1,4,0,1,2,1
3,4,Female,Loyal Customer,20,Personal Travel,Eco,2249,5,5,1,...,1,4,1,4,1,5,0,0,0,1
4,5,Male,disloyal Customer,38,Business travel,Eco,1995,2,2,2,...,5,2,5,3,1,2,0,110,102,0


In [14]:
def resumetable(df):
    print(f'데이터 세트 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': '피처'})
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고윳값 개수'] = df.nunique().values
    summary['첫 번째 값'] = df.loc[0].values
    summary['두 번째 값'] = df.loc[1].values
    summary['세 번째 값'] = df.loc[2].values

    return summary

resumetable(train)

데이터 세트 형상: (2999, 24)


,피처,데이터 타입,결측값 개수,고윳값 개수,첫 번째 값,두 번째 값,세 번째 값
0,id,int64,0,2999,1,2,3
1,Gender,object,0,2,Male,Male,Male
2,Customer Type,object,0,2,Loyal Customer,Loyal Customer,disloyal Customer
3,Age,int64,0,73,72,35,37
4,Type of Travel,object,0,2,Business travel,Business travel,Business travel
5,Class,object,0,3,Business,Business,Eco
6,Flight Distance,int64,0,2003,1784,2228,1970
7,Seat comfort,int64,0,6,1,2,4
8,Departure/Arrival time convenient,int64,0,6,1,5,3
9,Food and drink,int64,0,6,4,5,4


In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 24 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   id                                 2999 non-null   int64 
 1   Gender                             2999 non-null   object
 2   Customer Type                      2999 non-null   object
 3   Age                                2999 non-null   int64 
 4   Type of Travel                     2999 non-null   object
 5   Class                              2999 non-null   object
 6   Flight Distance                    2999 non-null   int64 
 7   Seat comfort                       2999 non-null   int64 
 8   Departure/Arrival time convenient  2999 non-null   int64 
 9   Food and drink                     2999 non-null   int64 
 10  Gate location                      2999 non-null   int64 
 11  Inflight wifi service              2999 non-null   int64 
 12  Inflig

In [20]:
train_x = train.drop(["id","target"],axis=1) 
train_y = train.target 

#라벨인코딩을 하기 위함 dictionary map 생성 함수
def make_label_map(dataframe):
    label_maps = {}
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            label_map = {'unknown':0}
            for i, key in enumerate(dataframe[col].unique()): #enumerate는 인덱스와 함께 고유값 반환, enumerate(['a', 'b', 'c'])는 (0, 'a'), (1, 'b'), (2, 'c')와 같은 튜플을 반환
                label_map[key] = i+1  
            label_maps[col] = label_map
    return label_maps

# 각 범주형 변수에 인코딩 값을 부여하는 함수
def label_encoder(dataframe, label_map):
    for col in dataframe.columns:
        if dataframe[col].dtype=='object':
            dataframe[col] = dataframe[col].map(label_map[col])
            dataframe[col] = dataframe[col].fillna(label_map[col]['unknown']) 
    return dataframe

# train 데이터 라벨 인코딩
label_map = make_label_map(train_x) # train 사용해 label map 생성
train_x = label_encoder(train_x, label_map) # train 라벨 인코딩

train_x.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,1,1,72,1,1,1784,1,1,4,1,...,5,2,2,3,2,5,2,0,0,0
1,1,1,35,1,1,2228,2,5,5,5,...,2,2,2,2,2,4,2,0,0,8
2,1,2,37,1,2,1970,4,3,4,1,...,1,1,3,2,5,1,4,0,1,2
3,2,1,20,2,2,2249,5,5,1,3,...,1,1,4,1,4,1,5,0,0,0
4,1,2,38,1,2,1995,2,2,2,2,...,5,5,2,5,3,1,2,0,110,102


In [21]:
resumetable(train_x)

데이터 세트 형상: (2999, 22)


,피처,데이터 타입,결측값 개수,고윳값 개수,첫 번째 값,두 번째 값,세 번째 값
0,Gender,int64,0,2,1,1,1
1,Customer Type,int64,0,2,1,1,2
2,Age,int64,0,73,72,35,37
3,Type of Travel,int64,0,2,1,1,1
4,Class,int64,0,3,1,1,2
5,Flight Distance,int64,0,2003,1784,2228,1970
6,Seat comfort,int64,0,6,1,2,4
7,Departure/Arrival time convenient,int64,0,6,1,5,3
8,Food and drink,int64,0,6,4,5,4
9,Gate location,int64,0,5,1,5,1


In [22]:
from sklearn.preprocessing import MinMaxScaler

num_features = ['Age','Flight Distance','Departure Delay in Minutes','Arrival Delay in Minutes']

scaler = MinMaxScaler()
train_x[num_features] = scaler.fit_transform(train_x[num_features]) 
train_x.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,1,1,0.890411,1,1,0.264656,1,1,4,1,...,5,2,2,3,2,5,2,0,0.000000,0.000000
1,1,1,0.383562,1,1,0.332619,2,5,5,5,...,2,2,2,2,2,4,2,0,0.000000,0.017021
2,1,2,0.410959,1,2,0.293127,4,3,4,1,...,1,1,3,2,5,1,4,0,0.002299,0.004255
3,2,1,0.178082,2,2,0.335833,5,5,1,3,...,1,1,4,1,4,1,5,0,0.000000,0.000000
4,1,2,0.424658,1,2,0.296954,2,2,2,2,...,5,5,2,5,3,1,2,0,0.252874,0.217021


In [25]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 학습 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

# 결과 저장을 위한 딕셔너리
results = {}

In [26]:
from sklearn.linear_model import LogisticRegression

# 모델 초기화 및 학습
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train, y_train)

# 예측
y_pred_logistic = logistic_model.predict(X_test)

# 평가
results['Logistic Regression'] = accuracy_score(y_test, y_pred_logistic)
print(f"Logistic Regression Accuracy: {results['Logistic Regression']}")
print(classification_report(y_test, y_pred_logistic))


Logistic Regression Accuracy: 0.8333333333333334
              precision    recall  f1-score   support

           0       0.82      0.83      0.82       282
           1       0.85      0.84      0.84       318

    accuracy                           0.83       600
   macro avg       0.83      0.83      0.83       600
weighted avg       0.83      0.83      0.83       600



In [28]:
from sklearn.tree import DecisionTreeClassifier

# 모델 초기화 및 학습
decision_tree_model = DecisionTreeClassifier(random_state=42)
decision_tree_model.fit(X_train, y_train)

# 예측
y_pred_tree = decision_tree_model.predict(X_test)

# 평가
results['Decision Tree'] = accuracy_score(y_test, y_pred_tree)
print(f"Decision Tree Accuracy: {results['Decision Tree']}")
print(classification_report(y_test, y_pred_tree))


Decision Tree Accuracy: 0.8816666666666667
              precision    recall  f1-score   support

           0       0.86      0.90      0.88       282
           1       0.90      0.87      0.89       318

    accuracy                           0.88       600
   macro avg       0.88      0.88      0.88       600
weighted avg       0.88      0.88      0.88       600



In [29]:
from sklearn.svm import SVC

# 모델 초기화 및 학습
svm_model = SVC(random_state=42)
svm_model.fit(X_train, y_train)

# 예측
y_pred_svm = svm_model.predict(X_test)

# 평가
results['SVM'] = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {results['SVM']}")
print(classification_report(y_test, y_pred_svm))


SVM Accuracy: 0.9016666666666666
              precision    recall  f1-score   support

           0       0.88      0.91      0.90       282
           1       0.92      0.89      0.91       318

    accuracy                           0.90       600
   macro avg       0.90      0.90      0.90       600
weighted avg       0.90      0.90      0.90       600



In [30]:
from sklearn.ensemble import RandomForestClassifier

# 모델 초기화 및 학습
random_forest_model = RandomForestClassifier(random_state=42)
random_forest_model.fit(X_train, y_train)

# 예측
y_pred_rf = random_forest_model.predict(X_test)

# 평가
results['Random Forest'] = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {results['Random Forest']}")
print(classification_report(y_test, y_pred_rf))


Random Forest Accuracy: 0.9116666666666666
              precision    recall  f1-score   support

           0       0.91      0.90      0.91       282
           1       0.91      0.92      0.92       318

    accuracy                           0.91       600
   macro avg       0.91      0.91      0.91       600
weighted avg       0.91      0.91      0.91       600



In [31]:
from sklearn.neural_network import MLPClassifier

# 모델 초기화 및 학습
neural_network_model = MLPClassifier(random_state=42, max_iter=1000)
neural_network_model.fit(X_train, y_train)

# 예측
y_pred_nn = neural_network_model.predict(X_test)

# 평가
results['Neural Network'] = accuracy_score(y_test, y_pred_nn)
print(f"Neural Network Accuracy: {results['Neural Network']}")
print(classification_report(y_test, y_pred_nn))


Neural Network Accuracy: 0.91
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       282
           1       0.95      0.88      0.91       318

    accuracy                           0.91       600
   macro avg       0.91      0.91      0.91       600
weighted avg       0.91      0.91      0.91       600



In [32]:
import lightgbm as lgb

# 데이터셋 준비
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# 파라미터 설정
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'verbose': -1,
    'random_state': 42
}

# 모델 학습
lgb_model = lgb.train(
    params, 
    lgb_train, 
    valid_sets=[lgb_eval], 
    num_boost_round=1000, 
    callbacks=[lgb.early_stopping(stopping_rounds=10)]
)

# 예측
y_pred_lgb = (lgb_model.predict(X_test) > 0.5).astype(int)

# 평가
results['LightGBM'] = accuracy_score(y_test, y_pred_lgb)
print(f"LightGBM Accuracy: {results['LightGBM']}")
print(classification_report(y_test, y_pred_lgb))


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[59]	valid_0's binary_logloss: 0.184593
LightGBM Accuracy: 0.92
              precision    recall  f1-score   support

           0       0.90      0.93      0.92       282
           1       0.94      0.91      0.92       318

    accuracy                           0.92       600
   macro avg       0.92      0.92      0.92       600
weighted avg       0.92      0.92      0.92       600



In [33]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# 모델 초기화 및 학습
xgb_model = xgb.XGBClassifier(eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

# 예측
y_pred_xgb = xgb_model.predict(X_test)

# 평가
results['XGBoost'] = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {results['XGBoost']}")
print(classification_report(y_test, y_pred_xgb))


XGBoost Accuracy: 0.9233333333333333
              precision    recall  f1-score   support

           0       0.91      0.93      0.92       282
           1       0.94      0.92      0.93       318

    accuracy                           0.92       600
   macro avg       0.92      0.92      0.92       600
weighted avg       0.92      0.92      0.92       600

